a bit DIY here
- [ ] for each star of interest
- [x] pull LCs from lightkurve
- [x] for each LC, do some simple iterative detrending & flare finding
- [x] mind the gaps
- [x] combine the close events
- [x] tune this for the data in hand, not in general
- [ ] save flare catalog/table for every star

hopefully can steal chunks of this code from the HelloTESS stuff?

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightkurve as lk
from astropy.table import Table
# from scipy.signal import savgol_filter
# from scipy.interpolate import CubicSpline

from stuff import *
import exoplanet as xo

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size':18})
matplotlib.rcParams.update({'font.family':'serif'})

In [3]:
FS = pd.read_csv('flare_sample_step1.csv')
FS

,KIC,TIC,Nfl,K1min,K30min,T2min
0,11143395,27236187,2,0.0,1.0,2.0
1,11495571,48504458,2,0.0,15.0,1.0
2,6117602,122136861,2,0.0,17.0,3.0
3,8408875,123229811,2,0.0,4.0,2.0
4,8682921,159047957,2,1.0,18.0,2.0
5,8429280,159721038,3,1.0,18.0,3.0
6,9153754,159761686,3,0.0,4.0,2.0
7,9201463,164458193,3,0.0,17.0,4.0
8,7505473,164463430,3,0.0,4.0,4.0
9,7341653,164644375,11,0.0,16.0,4.0


In [4]:
# let's start with k=5 (KIC 8429280), since it has 1min, 30min, and 2min

In [5]:
# len(lk.search_lightcurve('TIC'+str(FS['TIC'].values[k]), mission='TESS', exptime=120))

In [6]:
k=5
j=2
lc = lk.search_lightcurve('TIC'+str(FS['TIC'].values[k]), mission='TESS', exptime=120)[j].download()



# make harsh quality cuts, and chop out a known bad window of time (might add more later)
AOK = (lc['quality'].data == 0) & ((lc['time'].value < 1347) | (lc['time'].value > 1350))
med = np.nanmedian(lc['pdcsap_flux'].value[AOK])
xtr = 5

# ACF w/ Exoplanet package
acf = xo.autocorr_estimator(lc['time'].value[AOK], lc['pdcsap_flux'].value[AOK] / med,
                            yerr=lc['pdcsap_flux_err'].value[AOK] / med,
                            min_period=0.07, max_period=27, max_peaks=2)
if len(acf['peaks']) > 0:
    ACF_1dt = acf['peaks'][0]['period']
    ACF_1pk = acf['autocorr'][1][np.where((acf['autocorr'][0] == acf['peaks'][0]['period']))[0]][0]

    scl = np.abs(np.nanmedian(np.diff(lc['time'].value)))
    s_window = int(ACF_1dt / scl / xtr)
else:
    s_window = 128

# window must be odd for some smooth approaches
if (s_window % 2) == 0:
    s_window -= 1
    
print(scl, s_window, ACF_1dt)


yerr = lc['pdcsap_flux_err'].value[AOK] / med
x = lc['time'].value[AOK]
y = (lc['pdcsap_flux'].value[AOK] - med)/med


# find flares
FM, xhat, yhat = iflare(x, y, yerr, s_window, npass=6)

# split the flares into start/stops
istart, istop = flaresplit(FM, dlim=5)

print()
for l in range(len(istart)):
    print( EquivDur(x,y,yerr, xhat,yhat, istart[l],istop[l]) )



0.0013888763342038146 169 1.1784584094348247

(1.42159551830076, 0.13338043363974111)
(2.0338344752222497, 0.18667425847979777)
(3.9937638586168696, 0.2456895724348937)
(5.021318638405559, 0.25740954627656143)
(60.07306815819256, 0.40615914934913633)
(9.76452954017937, 0.2063729196815664)


In [7]:
# Nshort = np.zeros(sum(m1))
# Nlong = np.zeros(sum(m1))
# Ntess = np.zeros(sum(m1))
# for k in range(sum(m1)):
#     Nshort[k] = len(lk.search_lightcurve(str(spK['KIC'][m1].values[k]), mission='Kepler', exptime=60))
#     Nlong[k] = len(lk.search_lightcurve(str(spK['KIC'][m1].values[k]), mission='Kepler', exptime=1800))
#     Ntess[k] = len(lk.search_lightcurve(str(spK['KIC'][m1].values[k]), mission='TESS', exptime=120))

In [8]:

# yhat = savgol_filter(y, s_window, 3) # window size 51, polynomial order 3

# plt.figure(figsize=(13,5))

# plt.plot(x,y)
# plt.plot(x,yhat, color='red')

# # plt.xlim(1731,1735)
# # plt.plot(x, y - yhat)

In [9]:
# # find flares
# FM, xhat, yhat = iflare(x, y, yerr, s_window, npass=6)

# # split the flares into start/stops
# istart, istop = flaresplit(FM, dlim=5)

# for k in range(len(istart)):
#     print(EquivDur(x,y,xhat,yhat,istart[k], istop[k]))
# #     print(x[istart[k]], x[istop[k]])

#     plt.figure()
#     plt.plot(x,y)
#     plt.xlim(x[istart[k]]-.1, x[istop[k]]+.1)

In [10]:
# for k in range(len(istart)):
    
#     # identify continuum regions in smooth data
#     durk = x[istop[k]] - x[istart[k]]
#     c1 = np.where((xhat >= (x[istart[k]] - (durk*1.5))) & 
#                   (xhat <= (x[istart[k]] - (durk*0.5)))
#                  )[0]
#     c2 = np.where((xhat <= (x[istop[k]] + (durk*2.))) & 
#                   (xhat >= (x[istop[k]] + (durk)))
#                  )[0]

#     fok = np.where((x >= (x[istart[k]] - (durk*1.5))) & 
#                    (x <= (x[istop[k]] + (durk*2.))))[0]
#     fok2 = np.where((x >= (x[istart[k]] - (durk*0.5))) & 
#                     (x <= (x[istop[k]] + (durk))))[0]
    
#     cs = CubicSpline(np.append(xhat[c1], xhat[c2]), np.append(yhat[c1], yhat[c2]))
#     smo = cs(x[fok2])
    
#     ED = np.trapz(y[fok2] - smo, x[fok2]*60*60*24.)
#     chisq = np.sum( ((y[fok2] - smo)/yerr[fok2])**2.) / np.size(fok2)
#     EDerr = np.sqrt(ED**2. / (np.size(fok2)) / chisq)
#     print(ED, EDerr)

# #     plt.figure()
# #     plt.plot(x,y)
# #     # plt.plot(xhat,yhat)
# #     plt.plot(xhat[c1], yhat[c1], lw=6)
# #     plt.plot(xhat[c2], yhat[c2], lw=6)
# #     plt.plot(x[fok], smo)
# #     plt.xlim(x[fok].min(), x[fok].max())
# #     plt.ylim(y[fok].min(), y[fok].max())
